Here we provide code to find groupings of nodes to which the z-site gates are applied. These functions are collated in src/generating_graphs.jl for use in other notebooks.

To start, let's count the number of nodes at a certain number of generations with coordination number z.

In [ ]:
function numnodes(z::Integer,gens::Integer)
    if gens==0
        return 1
    else
        return Int64(1+z+z*(z-1)*((z-1)^(gens-1)-1)/(z-2))
    end
end

In [ ]:
numnodes(3,4)

Then we define a function which describes the connectivity of the graph, i.e. each pair defines a bond.

In [ ]:
function graph_nodes(z::Integer,gens::Integer)

    # #make first generation
    pairs=[[1,i] for i in 2:numnodes(z,1)]

    for gen in 2:gens
        count=numnodes(z,gen-1)+1
        for i in (numnodes(z,gen-2)+1):numnodes(z,gen-1)
            for j in 1:(z-1)
                push!(pairs,[i,count])
                count+=1
            end
        end

    end

    return pairs

end

In [ ]:
graph_nodes(3,4)

Now we split the bonds (ie pairs of nodes) according to the 2-coloring.

In [ ]:
function graph_nodes_2coloring(z::Integer,gens::Integer)

    orange=[[1,i] for i in 2:z]
    green=[[1,z+1]]

    
    for gen in 2:gens
        count=numnodes(z,gen-1)+1
        for i in (numnodes(z,gen-2)+1):numnodes(z,gen-1) #loop around the outside of the tree
            # println(i)
            #check colour of existing bond on i
            if any(i in p for p in orange)
                push!(green,[i,count])
                # println("green",count)
            else
                push!(orange,[i,count])
                # println("orange")
            end
            count+=1

            for j in 1:(z-2) #add z-2 new sites of the same colour
                if gen%2==0
                    push!(green,[i,count])
                    count+=1
                else
                    push!(orange,[i,count])
                    count+=1
                end
            end
            
        end

    end

    return orange,green

end

In [ ]:
graph_nodes_2coloring(3,4)

Next, group nodes into sets of z sites on which the z-site unitaries act. For simplicity in using this to apply z-site gates, we remove the single 'outlier' group at the edge which will contain only two nodes.

In [ ]:
function merge(pairs)
    zsets=[pairs[1]]
    for pair in pairs[2:end]
        count=0
        for i in 1:length(zsets)
            if !isempty(intersect(zsets[i], pair))
                zsets[i]=union(zsets[i],pair) 
                count+=1
            end
        end
        if count==0
            push!(zsets,pair)
        end
    end
    return zsets
end

function remove_pair(sets)
    return filter(x -> length(x) != 2, sets)
end

#find the sets of nodes on which the gate acts 
function graph_nodes_zsites(z,gens)
    o,g=graph_nodes_2coloring(z,gens)
    om=merge(o)
    gm=merge(g)
    # return om, gm
    return remove_pair(om),remove_pair(gm)
    
end

In [ ]:
graph_nodes_zsites(3,2)

Finally, we have a function which will find the set of nodes inside the geometric light cone after a certain time.

In [ ]:
function find_lc_zsite(z,gens,time)
    #finds the sites in the light cone from the origin at certain time
    trip_lists=graph_nodes_zsites(z,gens)
    lightcone=[1]

    for t in 1:time
        trips=trip_lists[(t-1)%2+1]
        for trip in trips
            if any(x in lightcone for x in trip)
                lightcone=union(lightcone,trip)
            end
        end
    end
    return lightcone
end

In [ ]:
find_lc_zsite(3,4,2)